In [3]:
!pip install poloniex

In [0]:
import poloniex
import time
import numpy as np
import math
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import rcParams
from matplotlib.ticker import *
%matplotlib inline

In [0]:
polo = poloniex.Poloniex()
# 変換可能な通貨一覧を取得
pairs = [pair for pair in polo.returnTicker()]

In [0]:
#データフレームの作成
currency_histrical_data = pd.DataFrame()

In [0]:
#サンプリングする時間の変数を設定
end_time = time.time()
num_data = 22000

#サンプリング間隔の設定(分)
sampling_period = 5
sampling_period_sec = sampling_period * 60

start_time = time.time()-num_data*sampling_period_sec

#現在時刻取得
current_time = datetime.datetime.now().strftime('%Y%m%d')

#取得する通貨情報一覧
convert_list_all = ['USDT_ETH','USDT_BTC','USDT_ETC','USDT_STR','USDT_DASH',
                'USDT_REP','USDT_LTC','USDT_XMR','USDT_BCH','USDT_NXT',
                'USDT_ZEC','USDT_XRP']
#USDT_BCHは欠損値が多いので、除外
convert_list = ['USDT_ETH','USDT_BTC','USDT_ETC','USDT_STR','USDT_DASH',
                'USDT_REP','USDT_LTC','USDT_XMR','USDT_NXT','USDT_ZEC','USDT_XRP']

#予測対象の通貨ペアの設定
target_pair = 'USDT_BTC'

save_file_train = './dataset/'+ target_pair + '_'+ str(sampling_period) + 'minutes_1year_'+ current_time + '.csv'

In [8]:
print(start_time, ":",end_time, ":", (end_time - start_time)/5/60)

1519470832.2829459 : 1526070832.2828765 : 21999.999999768734


In [0]:
chart_frame

,close,date,high,low,open,quoteVolume,volume,weightedAverage,date_date
0,462.86232997,1512186600,463.97186323,461.99999999,461.99999999,132.13966638,61052.40099115,462.02932596,2017-12-02 12:50:00
1,460.00000002,1512186900,462.86232999,460.00000001,462.86232999,128.28649698,59143.13515793,461.02385325,2017-12-02 12:55:00
2,460,1512187200,460.54568942,460,460.00000002,178.04536006,81904.04652801,460.01786567,2017-12-02 13:00:00
3,457.51600031,1512187500,460,457.34860009,460,290.72215405,133327.41984219,458.60770493,2017-12-02 13:05:00
4,456.19493531,1512187800,458.00000025,455.50338918,458.00000025,306.92536931,140242.10035989,456.92573629,2017-12-02 13:10:00
5,452.99000004,1512188100,456.19493522,452.6752512,455.50339012,902.87542252,410401.51109976,454.54943269,2017-12-02 13:15:00
6,452.89999997,1512188400,454,450.34573981,454,238.63478937,107806.22904387,451.76241623,2017-12-02 13:20:00
7,451,1512188700,452.89323939,450.09992174,452.49999995,223.92658325,100990.45805285,450.99807529,2017-12-02 13:25:00
8,454.00000001,1512189000,454.00000001,451.65000001,452.89323834,519.28516364,235236.77560312,453.00114864,2017-12-02 13:30:00
9,455.79490944,1512189300,456.1949353,453.00000001,454.00000004,145.08452046,66138.67952287,455.86310181,2017-12-02 13:35:00


In [0]:
# 指定したサンプリング間隔で120日分のデータを読み込む
for each_currency in convert_list:
    chart_data = polo.returnChartData(each_currency, period=sampling_period_sec, start=start_time, end=end_time)    
    chart_frame = pd.DataFrame(chart_data)
    chart_frame['date_date'] = chart_frame.date.apply(lambda x:datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))
    currency_histrical_data[each_currency + "_close"]=chart_frame['close']
    currency_histrical_data[each_currency + "_high"]=chart_frame['high']
    currency_histrical_data[each_currency + "_low"]=chart_frame['low']
    currency_histrical_data[each_currency + "_open"]=chart_frame['open']
    currency_histrical_data[each_currency + "_volume"]=chart_frame['volume']
    time.sleep(1) #IPがbanされないように、1秒間sleep
currency_histrical_data['DATE_DATE'] = chart_frame['date_date']

In [0]:
currency_histrical_data.to_csv(save_file_train, index=False, encoding='UTF-8_sig')

In [0]:
#データの読み込み
currency_hist_data = pd.read_csv(save_file_train, encoding='UTF-8')
#日付のDATE型に変更しておく
currency_hist_data['DATE'] = currency_hist_data.DATE_DATE.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
currency_hist_data = currency_hist_data.drop(["DATE_DATE"], axis=1)
#データ加工用にdateをindex化
currency_hist_data2 = currency_hist_data.set_index(['DATE'])

In [0]:
#通貨ごとにSMAと±2σのボリンジャーバンドの値、および前の観測タイミングと比較した傾きを変数として追加

#移動平均の算出
def calc_moving_average(baseday, delta, target):
    day_before_20 = baseday - datetime.timedelta(minutes=delta)
    try:
        #一日毎の値を取得
        time_delta_data = currency_hist_data2.loc[(day_before_20):(baseday)]
    except:
        return np.NaN
    
    return  time_delta_data[target + "_close"].mean()

#ボリンジャーバンド算出
def calc_BB(baseday, delta, target, target_sigma, SMA):
    day_before_20 = baseday - datetime.timedelta(minutes=delta)
    try:
        #一日毎の値を取得
        time_delta_data = currency_hist_data2.loc[(day_before_20):(baseday)]
    except:
        return np.NaN
    
    std_ = time_delta_data[target + "_close"].std()
    return 0 if math.isnan(std_) else SMA + target_sigma*std_

In [0]:
sampling_delta = int(5*sampling_period)
for each_currency in convert_list:
    currency_hist_data[each_currency + '_SMA'] = currency_hist_data.apply(lambda x:calc_moving_average(x.DATE, sampling_delta, each_currency), axis=1)
    currency_hist_data[each_currency + '_plus2sigma'] = currency_hist_data.apply(lambda x:calc_BB(x.DATE, sampling_delta, each_currency, 2, x[each_currency + '_SMA']), axis=1)
    currency_hist_data[each_currency + '_plus1sigma'] = currency_hist_data.apply(lambda x:calc_BB(x.DATE, sampling_delta, each_currency, 1, x[each_currency + '_SMA']), axis=1)
    currency_hist_data[each_currency + '_minus1sigma'] = currency_hist_data.apply(lambda x:calc_BB(x.DATE, sampling_delta, each_currency, -1, x[each_currency + '_SMA']), axis=1)
    currency_hist_data[each_currency + '_minus2sigma'] = currency_hist_data.apply(lambda x:calc_BB(x.DATE, sampling_delta, each_currency, -2, x[each_currency + '_SMA']), axis=1)

In [0]:
#データ加工用にdateをindex化
currency_hist_data2 = currency_hist_data.set_index(['DATE'])

#5分前とのdiffを取る
def get_diff(baseday, delta, diff_value, target_column):
    targetday = baseday - datetime.timedelta(minutes=delta)
    try:
        targetday_list = currency_hist_data2.loc[(targetday)]
    except:
        return np.NaN
    return float(diff_value) - float(targetday_list[target_column])

In [0]:
for each_currency in convert_list:
    currency_hist_data[each_currency + '_open_diff'] = currency_hist_data.apply(lambda x:get_diff(x.DATE, sampling_period, x[each_currency + '_open'], each_currency + '_open'), axis=1)
    currency_hist_data[each_currency + '_high_diff'] = currency_hist_data.apply(lambda x:get_diff(x.DATE, sampling_period, x[each_currency + '_high'], each_currency + '_high'), axis=1)
    currency_hist_data[each_currency + '_low_diff'] = currency_hist_data.apply(lambda x:get_diff(x.DATE, sampling_period, x[each_currency + '_low'], each_currency + '_low'), axis=1)
    currency_hist_data[each_currency + '_close_diff'] = currency_hist_data.apply(lambda x:get_diff(x.DATE, sampling_period, x[each_currency + '_close'], each_currency + '_close'), axis=1)
    currency_hist_data[each_currency + '_volume_diff'] = currency_hist_data.apply(lambda x:get_diff(x.DATE, sampling_period, x[each_currency + '_close'], each_currency + '_volume'), axis=1)
    currency_hist_data[each_currency + '_SMA_diff'] = currency_hist_data.apply(lambda x:get_diff(x.DATE, sampling_period, x[each_currency + '_SMA'], each_currency + '_SMA'), axis=1)
    currency_hist_data[each_currency + '_plus2sigma_diff'] = currency_hist_data.apply(lambda x:get_diff(x.DATE, sampling_period, x[each_currency + '_plus2sigma'], each_currency + '_plus2sigma'), axis=1)
    currency_hist_data[each_currency + '_plus1sigma_diff'] = currency_hist_data.apply(lambda x:get_diff(x.DATE, sampling_period, x[each_currency + '_plus1sigma'], each_currency + '_plus1sigma'), axis=1)
    currency_hist_data[each_currency + '_minus1sigma_diff'] = currency_hist_data.apply(lambda x:get_diff(x.DATE, sampling_period, x[each_currency + '_minus1sigma'], each_currency + '_minus1sigma'), axis=1)
    currency_hist_data[each_currency + '_minus2sigma_diff'] = currency_hist_data.apply(lambda x:get_diff(x.DATE, sampling_period, x[each_currency + '_minus2sigma'], each_currency + '_minus2sigma'), axis=1)  

In [0]:
#目的変数の設定
def get_price_after_delta(baseday, delta_minutes, target):
    targetday = baseday + datetime.timedelta(minutes=delta_minutes)
    try:
        targetday_list = currency_hist_data2.loc[(targetday)]
    except:
        return np.NaN
    return targetday_list[target + "_SMA"]
    
    
currency_hist_data[target_pair + '_SMA_after_' + str(sampling_period) + 'min'] = currency_hist_data.apply(lambda x:get_price_after_delta(x.DATE, sampling_period, target_pair), axis=1)

In [0]:
#現在と30分後の金額の差分を取る
currency_hist_data[target_pair + '_SMA_after_' + str(sampling_period) + 'min_diff'] = currency_hist_data.apply(lambda x: x[target_pair + '_SMA_after_' + str(sampling_period) + 'min'] - x[target_pair + '_SMA'], axis=1)
#上がったか下がったかの０１判定
currency_hist_data[target_pair + '_SMA_after_' + str(sampling_period) + 'min_flag'] = currency_hist_data.apply(lambda x: 1 if x[target_pair + '_SMA_after_' + str(sampling_period) + 'min_diff'] > 0 else 0, axis=1)

#現在と30分後の金額の比率を取る
currency_hist_data[target_pair + '_SMA_after_' + str(sampling_period) + 'min_ratio'] = currency_hist_data.apply(lambda x: x[target_pair + '_SMA_after_' + str(sampling_period) + 'min']/x[target_pair + '_SMA'], axis=1)

#比率の自然対数もとる
currency_hist_data[target_pair + '_SMA_after_' + str(sampling_period) + 'min_log'] = currency_hist_data.apply(lambda x: math.log(x[target_pair + '_SMA_after_' + str(sampling_period) + 'min_ratio']), axis=1)

In [0]:
#加工済みデータの保存
save_file_train2 = './dataset/'+ target_pair + '_' + str(sampling_period) + 'minutes_1year_' + current_time + '_1.csv'

currency_hist_data.to_csv(save_file_train2, index=False, encoding='UTF-8_sig')

In [0]:
#plt.hist(currency_hist_data[target_pair + '_close_after_' + str(sampling_period) + 'min_ratio'],range=(0.85, 1.15), bins=50)

In [0]:
#データの読み込み
coin_data = pd.read_csv(save_file_train2, encoding='UTF-8')
coin_data['DATE'] = coin_data.DATE.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [0]:
date_border = coin_data.loc[0].DATE + datetime.timedelta(days=20)
coin_data_ = coin_data[coin_data.DATE >= date_border]

In [0]:
save_file_train3 = './dataset/'+ target_pair + '_' + str(sampling_period) + 'minutes_1year_' + current_time + '_2.csv'
coin_data_.to_csv(save_file_train3, index=False, encoding='UTF-8_sig')

In [0]:
save_file_train4 = './dataset/USDT_BTC_5minutes_1year_20180130_2.csv'
coin_data = pd.read_csv(save_file_train4, encoding='UTF-8')

In [0]:
coin_data = pd.read_csv(save_file_train4, encoding='UTF-8')
coin_data['DATE'] = coin_data.DATE.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [0]:
coin_data2 = coin_data.set_index(['DATE'])

In [0]:
#目的変数の設定
def get_price_after_delta(baseday, delta_minutes, target):
    targetday = baseday + datetime.timedelta(minutes=delta_minutes)
    try:
        targetday_list = coin_data2.loc[(targetday)]
    except:
        return np.NaN
    return targetday_list[target + "_SMA"]
    
    
coin_data[target_pair + '_SMA_after_' + str(sampling_period) + 'min'] = coin_data.apply(lambda x:get_price_after_delta(x.DATE, sampling_period, target_pair), axis=1)

In [0]:
coin_data.head()

,USDT_ETH_close,USDT_ETH_high,USDT_ETH_low,USDT_ETH_open,USDT_BTC_close,USDT_BTC_high,USDT_BTC_low,USDT_BTC_open,USDT_ETC_close,USDT_ETC_high,...,USDT_XRP_plus2sigma_diff,USDT_XRP_plus1sigma_diff,USDT_XRP_minus1sigma_diff,USDT_XRP_minus2sigma_diff,USDT_BTC_close_after_5min,USDT_BTC_close_after_5min_diff,USDT_BTC_close_after_5min_flag,USDT_BTC_close_after_5min_ratio,USDT_BTC_close_after_5min_log,USDT_BTC_SMA_after_5min
0,368.500003,369.200000,368.500000,369.200000,8261.983652,8261.983652,8245.100000,8245.100000,18.320001,18.400000,...,7.174433e-05,4.902645e-05,3.590691e-06,-1.912719e-05,8250.070340,-11.913312,0,0.998558,-0.001443,8215.915246
1,369.000001,369.900000,368.500003,368.592100,8250.070340,8263.762592,8247.000000,8261.983652,18.400053,18.430000,...,3.313448e-08,2.156724e-08,-1.567241e-09,-1.313448e-08,8257.000000,6.929660,1,1.000840,0.000840,8219.105722
2,368.490000,369.110000,368.000763,369.110000,8257.000000,8263.791295,8247.070000,8256.999999,18.451000,18.484079,...,1.102174e-04,7.742082e-05,1.182775e-05,-2.096879e-05,8238.000001,-18.999999,0,0.997699,-0.002304,8221.772388
3,367.300000,368.490000,367.295698,368.490000,8238.000001,8263.000000,8233.248028,8257.000000,18.490000,18.490000,...,1.099794e-04,8.154114e-05,2.466457e-05,-3.773709e-06,8210.000000,-28.000001,0,0.996601,-0.003405,8222.343817
4,366.310000,367.999999,366.000000,367.295698,8210.000000,8245.000000,8209.526356,8240.000000,18.430000,18.490070,...,1.007707e-04,7.408724e-05,2.072038e-05,-5.963048e-06,8219.000000,9.000000,1,1.001096,0.001096,8224.200959


In [0]:
#現在と30分後の金額の差分を取る
coin_data[target_pair + '_SMA_after_' + str(sampling_period) + 'min_diff'] = coin_data.apply(lambda x: x[target_pair + '_SMA_after_' + str(sampling_period) + 'min'] - x[target_pair + '_SMA'], axis=1)
#上がったか下がったかの０１判定
coin_data[target_pair + '_SMA_after_' + str(sampling_period) + 'min_flag'] = coin_data.apply(lambda x: 1 if x[target_pair + '_SMA_after_' + str(sampling_period) + 'min_diff'] > 0 else 0, axis=1)

In [0]:
save_file_train4 = './dataset/USDT_BTC_5minutes_1year_20180130_3.csv'
coin_data.to_csv(save_file_train4, index=False, encoding='UTF-8_sig')